## Local Linear Forests

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor

class LocalLinearForestRegressor(RandomForestRegressor):
    def __init__(self,
                 n_estimators=100,
                 criterion='squared_error',
                 max_depth=None,
                 min_samples_split=4,
                 min_samples_leaf=2,
                 min_weight_fraction_leaf=0.,
                 max_features=0.7,
                 max_leaf_nodes=None,
                 min_impurity_decrease=0.,
                 bootstrap=True,
                 oob_score=True,
                 n_jobs=-1,
                 random_state=None,
                 verbose=0,
                 warm_start=False,
                 max_samples=0.5):
        super().__init__(n_estimators=n_estimators,
                         criterion=criterion,
                         max_depth=max_depth,
                         min_samples_split=min_samples_split,
                         min_samples_leaf=min_samples_leaf,
                         min_weight_fraction_leaf=min_weight_fraction_leaf,
                         max_features=max_features,
                         max_leaf_nodes=max_leaf_nodes,
                         min_impurity_decrease=min_impurity_decrease,
                         bootstrap=bootstrap,
                         oob_score=oob_score,
                         n_jobs=n_jobs,
                         random_state=random_state,
                         verbose=verbose,
                         warm_start=warm_start,
                         max_samples=max_samples)

        self._incidence_matrix = None
        self._X_train = None
        self._Y_train = None

    def _extract_leaf_nodes_ids(self, X):
        leafs = [e.apply(X).reshape(-1, 1) for e in self.estimators_]
        leaf_nodes_ids = np.concatenate(leafs, axis=1)
        assert leaf_nodes_ids.shape[0] == X.shape[0]
        assert leaf_nodes_ids.shape[1] == len(self.estimators_)
        return leaf_nodes_ids

    def fit(self, X_split, y, sample_weight=None):
        super().fit(X_split, y, sample_weight=sample_weight)
        self._X_train = X_split
        self._Y_train = y
        self._incidence_matrix = self._extract_leaf_nodes_ids(X_split)
        return self

    def _get_forest_coefficients(self, observation_leaf_ids):
        # Initialize an array to hold the coefficients
        coeffs = np.zeros(self._X_train.shape[0])

        # Iterate over each tree (column in the incidence matrix)
        for j in range(observation_leaf_ids.shape[1]):
            # Find the samples in the training set that match the leaf node of the test observation
            matching_nodes = (self._incidence_matrix[:, j] == observation_leaf_ids[0, j])

            # Calculate the counts for the matching nodes
            counts = np.sum(matching_nodes)

            # If counts are greater than 0, update the coefficients
            if counts > 0:
                coeffs += matching_nodes / counts

        # Normalize by the number of estimators (trees)
        return coeffs / self.n_estimators


    def predict_LLF(self, X_test, lam=0.1):
        results = []
        X_test = np.array(X_test)

        for i in range(X_test.shape[0]):
            x0 = X_test[i, :].reshape(1, -1)
            actual_leaf_ids = self._extract_leaf_nodes_ids(x0)
            alpha_i = self._get_forest_coefficients(actual_leaf_ids)

            # Centered regression matrix with intercept
            Delta = np.hstack([np.ones((self._X_train.shape[0], 1)), self._X_train - x0])

            # Diagonal matrix A
            A = np.diag(alpha_i)

            # Diagonal matrix J
            d = self._X_train.shape[1]
            J = np.diag([0] + [1] * d)

            # Solve the linear system
            inv_term = np.linalg.inv(Delta.T @ A @ Delta + lam * J)
            theta_hat = inv_term @ Delta.T @ A @ self._Y_train

            # Prediction for x0
            u_hat = theta_hat[0]
            results.append(u_hat)

        return np.array(results).reshape(-1)


## Random Forest

In [2]:
def train_test_rf(X_train, Y_train, X_test, n_estimators = 500, max_depth = 10):
    forest = RandomForestRegressor(n_estimators= n_estimators, max_depth=max_depth, min_samples_split=4, min_samples_leaf = 2)
    forest.fit(X_train, Y_train)
    predictions = forest.predict(X_test)

    return predictions

## XGBoost

In [62]:
def predict_xgboost(X_train, Y_train, X_test, ntrees_max=500, max_depth=10, learning_rate=0.5, subsample=0.5, colsample_bytree=0.5, reg_lambda=10, objective='reg:squarederror'):
    xg_boost = xgb.XGBRegressor(
        n_estimators=ntrees_max,
        max_depth=max_depth,
        learning_rate=learning_rate,
    )
    # No early stopping and using all training data
    xg_boost.fit(X_train, Y_train, verbose=False)

    xg_preds = xg_boost.predict(X_test)
    return xg_preds


## Bayesian Additive Regression Trees

In [5]:
%pip install git+https://github.com/JakeColtman/bartpy.git --no-deps

from IPython.utils import io
def predict_bart(X_train, Y_train, X_test, n_trees=50, n_chains=4, n_burn=20, n_samples=200):
    with io.capture_output() as captured:
        bart_model = SklearnModel(n_jobs=1, n_chains=n_chains, n_trees=n_trees, n_burn=n_burn, n_samples=n_samples)
        bart_model.fit(X_train, Y_train)
        bart_preds = bart_model.predict(X_test)
    return bart_preds

  Cloning https://github.com/JakeColtman/bartpy.git to c:\users\jnoot\appdata\local\temp\pip-req-build-7k7cgh9l
  Resolved https://github.com/JakeColtman/bartpy.git to commit 09e409e91dd1b9d44784c788c2d731dafb181eb0
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/JakeColtman/bartpy.git 'C:\Users\JNoot\AppData\Local\Temp\pip-req-build-7k7cgh9l'


## Lasso Random Forest

In [6]:
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def predict_lassoRF(X, Y, X_test, n_estimators=200, max_depth=10):
    # Split the data into two halves
    n = len(Y)
    inds = np.random.choice(range(n), size=n//2, replace=False)
    inds_not = np.array([i for i in range(n) if i not in inds])

    # Train Lasso on the first half of the data
    lasso = LassoCV(cv=2, max_iter=50, tol=1e-2)
    lasso.fit(X[inds], Y[inds])
    lasso_preds = lasso.predict(X[inds_not])
    lasso_resids = Y[inds_not] - lasso_preds

    # Train Random Forest on the second half of the data using the residuals from Lasso
    rf = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split = 4, min_samples_leaf = 2, oob_score = True)
    rf.fit(X[inds_not], lasso_resids)

    # Predict on the test set
    rf_preds = rf.predict(X_test)
    lasso_preds_test = lasso.predict(X_test)

    return rf_preds + lasso_preds_test


## Simulation 1 - Data Generating Process

In [68]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from bartpy.sklearnmodel import SklearnModel
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm

def generate_GJR(n,omega, alpha, gamma, beta, mu, sigma):
  # Simulate GJR-GARCH(1,1) process
  np.random.seed()
  residuals = np.random.normal(scale=sigma, size=n)
  volatility = np.zeros(n)
  returns = np.zeros(n)

  # Initial volatility
  volatility[0] = np.sqrt(omega / (1 - alpha - gamma / 2 - beta))

  for t in range(1, n):
      # GJR-GARCH process
      indicator = (residuals[t-1] < 0).astype(float)
      volatility[t] = np.sqrt(omega + (alpha + gamma * indicator) * (residuals[t-1] ** 2) + beta * (volatility[t-1] ** 2))

      # Generate returns
      returns[t] = mu + volatility[t] * residuals[t]

  return returns, volatility

# Generate lagged features for forecasting
def create_lagged_features(data, lag=1):
    lagged_data = np.column_stack([np.roll(data, i) for i in range(1, lag+1)])
    return lagged_data[lag:]

def generate_features(returns, volatility, lag=5, p = 10):
    X_lagged = create_lagged_features(returns, lag)
    if p > 0:
        random_features = np.random.rand(X_lagged.shape[0], p)  # Ensure matching rows
        X = np.hstack((X_lagged, random_features))
    else:
        X = X_lagged
    y = volatility[lag:]
    return X, y

# Simulation and forecasting
def simulation_run(n, p, sigma, omega, alpha, gamma, beta, mu, num_reps=50, lag=5):
    errors_list = []
    for _ in tqdm(range(num_reps)):
        returns, volatility = generate_GJR(n, omega, alpha, gamma, beta, mu, sigma)
        X, y = generate_features(returns, volatility, lag, p)
        train_size = len(X)-1

        # Train-test split
        X_train, X_test = X[:train_size], X[train_size:]
        Y_train, y_test = y[:train_size], y[train_size:]

        #Random Forest
        RF_predictions = train_test_rf(X_train, Y_train, X_test, n_estimators = 500, max_depth = 7)
        RF_mse = mean_squared_error(y_test, RF_predictions)

        #Lasso Random Forest
        LRF_preds = predict_lassoRF(X_train, Y_train, X_test, n_estimators = 500, max_depth = 7)
        LRF_mse = mean_squared_error(y_test, LRF_preds)

        #Local Linear Forest
        LLF = LocalLinearForestRegressor(n_estimators = 500, max_depth = 7)
        LLF.fit(X_train, Y_train)
        LLF_predictions = LLF.predict_LLF(X_test)
        LLF_mse = mean_squared_error(y_test, LLF_predictions)

        #Bayesian Additive Regression Trees
        BART_predictions = predict_bart(X_train, Y_train, X_test)
        BART_mse = mean_squared_error(y_test, BART_predictions)

        #XGBoost
        XG_predictions = predict_xgboost(X_train, Y_train, X_test, ntrees_max = 500, max_depth = 7)
        XG_mse = mean_squared_error(y_test, XG_predictions)

        # Collect errors
        errors = {
            "LLF": LLF_mse,
            "RF": RF_mse,
            "Lasso RF": LRF_mse,
            "BART": BART_mse,
            "XGBoost": XG_mse
        }
        errors_list.append(errors)

    mean_errors = {model: np.mean([errors[model] for errors in errors_list]) for model in errors_list[0]}
    return mean_errors

# Simulation parameters
num_reps = 50
efficient_run = True

# Parameters for GJR-GARCH(1,1) model
omega = 0.0000908
alpha = 0.03569
beta = 0.87636
gamma = 0.06178693
mu= -0.000215
lags = 5

ns = [1000, 5000]
ps = [0,5,10]
sigmas = [1,5,10]

if efficient_run:
  ns = [5000]
  ps = [10]
  sigmas = [10]

args = [(n, p, sigma) for n in ns for p in ps for sigma in sigmas]
full_results = []
for arguments in args:
    print(arguments)
    mses = simulation_run(*arguments, omega, alpha, gamma, beta, mu, num_reps, lags)
    #print([*np.round(np.sqrt(mses), 3)])
    full_results.append([*np.round(np.sqrt(list(mses.values())), 3)])
    print([*np.round(np.sqrt(list(mses.values())), 3)])

full_results = np.array(full_results)

print(full_results)

(5000, 10, 10)


100%|██████████| 30/30 [25:51<00:00, 51.71s/it]

[0.729, 0.742, 0.734, 0.728, 0.554]
[[0.729 0.742 0.734 0.728 0.554]]


## Simulation 2 & 3 - Friedman and Smoothness

In [75]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from bartpy.sklearnmodel import SklearnModel
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from tqdm import tnrange, notebook

def friedman(x):
    return 10 * np.sin(np.pi * x[0] * x[1]) + 20 * ((x[3] - 0.5) ** 2) + 10 * x[4] + 5 * x[5]

def smoothness(x):
    return np.log(1 + np.exp(6 * x[0]))

def simulation_run(function, n, p, sigma, num_reps=50, num_test = 1000):
    errors = []
    for _ in tqdm(range(num_reps)):
      #Simulate
      X_train = np.random.rand(n, p)
      Y_train = np.apply_along_axis(function, 1, X_train) + sigma * np.random.normal(size=n)
      X_test = np.random.rand(num_test, p)
      truth = np.apply_along_axis(function, 1, X_test)

      #Random Forest
      RF_predictions = train_test_rf(X_train, Y_train, X_test, n_estimators = 200, max_depth = 10)
      RF_mse = mean_squared_error(truth, RF_predictions)

      #Lasso Random Forest
      LRF_preds = predict_lassoRF(X_train, Y_train, X_test, n_estimators = 200, max_depth = 10)
      LRF_mse = mean_squared_error(truth, LRF_preds)

      #Local Linear Forest
      LLF = LocalLinearForestRegressor(n_estimators = 200, max_depth = 10)
      LLF.fit(X_train, Y_train)
      LLF_predictions = LLF.predict_LLF(X_test)
      LLF_mse = mean_squared_error(truth, LLF_predictions)

      #Bayesian Additive Regression Trees
      BART_predictions = predict_bart(X_train, Y_train, X_test)
      BART_mse = mean_squared_error(truth, BART_predictions)

      #XGBoost
      XG_predictions = predict_xgboost(X_train, Y_train, X_test, ntrees_max = 200, max_depth = 10)
      XG_mse = mean_squared_error(truth, XG_predictions)

      #Errors
      errors.append([LLF_mse, RF_mse, LRF_mse, BART_mse, XG_mse])

    return np.mean(errors, axis=0)

efficient_run = True
num_reps = 30
func = "smoothness"

if func == "friedman":
  function = friedman
  ps = [10,30,50]
  ns = [1000, 5000]
  sigmas = [5, 20]

  if efficient_run:
      ps = [30]
      ns = [5000]
      sigmas = [20]

if func == "smoothness":
  function = smoothness
  ps = [5,20]
  ns = [1000, 5000]
  sigmas = [0.1,1,2]

  if efficient_run:
    ps = [5]
    ns = [5000]
    sigmas = [2]

args = [(n, p, sigma) for n in ns for p in ps for sigma in sigmas]
full_results = []
for arguments in args:
    print(arguments)
    mses = simulation_run(function, *arguments, num_reps)
    full_results.append([*np.round(np.sqrt(mses), 3)])
    print([*np.round(np.sqrt(mses), 3)])

full_results = np.array(full_results)

print(full_results)

(5000, 5, 2)


100%|██████████| 30/30 [1:34:14<00:00, 188.49s/it]

[0.246, 0.36, 0.294, 0.223, 1.196]
[[0.246 0.36  0.294 0.223 1.196]]


## Simulation 4 - Adversarial Setting

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tnrange, notebook

def adverserial(x):
    return 10/ (1 + np.exp(-10 * (x[0] - 0.5))) + 5/ (1 + np.exp(-10 * (x[1] - 0.5)))

def confidence_interval(predictions, variances):
    lower_bounds = predictions - 1.96 * np.sqrt(variances)
    upper_bounds = predictions + 1.96 * np.sqrt(variances)
    return lower_bounds, upper_bounds

def get_variance(model, X_train, Y_train, X_test, n_estimators=25, subsample_size=0.5):
    #Bootstrap_of_little_bags
    n_samples = X_train.shape[0]
    bag_size = int(n_samples * subsample_size)

    predictions = np.zeros((X_test.shape[0], n_estimators))

    for i in range(n_estimators):
        # Create a bootstrap sample
        indices = np.random.choice(n_samples, bag_size, replace=True)
        X_subsample = X_train[indices]
        Y_subsample = Y_train[indices]

        if model == "RF":
          predictions[:, i] = train_test_rf(X_subsample, Y_subsample, X_test)
        elif model == "LLF":
          predictions[:,i] = predict_LLF(X_train, Y_train, X_test)

    # Calculate the variance of the predictions
    prediction_variance = np.var(predictions, axis=1)

    return prediction_variance

def simulation_run(n, p, sigma, num_reps=50, num_test = 1000):
    errors = []
    coverages_rf = []
    interval_lengths_rf = []
    coverages_llf = []
    interval_lengths_llf = []

    for _ in notebook.tqdm(range(num_reps)):
      #Simulate
      X_train = np.random.rand(n, p)
      Y_train = np.apply_along_axis(adverserial, 1, X_train) + sigma * np.random.normal(size=n)
      X_test = np.random.rand(num_test, p)
      truth = np.apply_along_axis(adverserial, 1, X_test)

      #Random Forest
      RF_predictions = train_test_rf(X_train, Y_train, X_test)
      variancesRF = get_variance("RF", X_train, Y_train, X_test)
      RF_lower, RF_upper = confidence_interval(RF_predictions, variancesRF)
      RF_mse = mean_squared_error(truth, RF_predictions)
      RF_coverage = np.mean((truth >= RF_lower) & (truth <= RF_upper))
      RF_interval_length = np.mean(RF_upper - RF_lower)

      #Local Linear Forest
      LLF_predictions = predict_LLF(X_train, Y_train, X_test)
      variancesLLF = get_variance("LLF", X_train, Y_train, X_test)
      LLF_lower, LLF_upper = confidence_interval(LLF_predictions, variancesLLF)
      LLF_mse = mean_squared_error(truth, LLF_predictions)
      LLF_coverage = np.mean((truth >= LLF_lower) & (truth <= LLF_upper))
      LLF_interval_length = np.mean(LLF_upper - LLF_lower)

      #for i in range(n):
        #print(truth[i], RF_lower[i], RF_upper[i])

      # Update Errors, Coverage and Interval lengths
      errors.append([RF_mse, LLF_mse])
      coverages_rf.append(RF_coverage)
      interval_lengths_rf.append(RF_interval_length)
      coverages_llf.append(LLF_coverage)
      interval_lengths_llf.append(LLF_interval_length)

    #Get average errors, coverages and interval lenghts
    avg_errors = np.mean(errors, axis=0)
    avg_coverages_rf = np.mean(coverages_rf)
    avg_interval_lengths_rf = np.mean(interval_lengths_rf)
    avg_coverages_llf = np.mean(coverages_llf)
    avg_interval_lengths_llf = np.mean(interval_lengths_llf)

    return avg_errors, avg_coverages_rf, avg_interval_lengths_rf, avg_coverages_llf, avg_interval_lengths_llf

efficient = True
ps = [5,20]
ns = [500,2000,10000]
sigmas = [5]

if(efficient == True):
   ps = [5,20]
   ns = [500,2000]

args = [(n, p, sigma) for n in ns for p in ps for sigma in sigmas]
full_results = []
for arguments in notebook.tqdm(args):
    print(arguments)
    mses, coverage_rf, length_rf, coverage_llf, length_llf = simulation_run(*arguments)
    full_results.append([*np.round(np.sqrt(mses), 3), coverage_rf, length_rf, coverage_llf, length_llf])

full_results = np.array(full_results)

print(full_results)


  0%|          | 0/4 [00:00<?, ?it/s]

(500, 5, 5)


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 